In [ ]:
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Scatter_Plot.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/QQ_Plot.ipynb

In [66]:

def Stacking_Regression(X_train, X_test, y_train, y_test, filtered_data, param_dict):
    ml_type = 'SL'

    # Define base learners
    rf_regressor = RandomForestRegressor(n_estimators=param_dict[0]['n_estimators'], 
                                         max_depth=param_dict[0]['max_depth'],
                                         min_samples_leaf=param_dict[0]['min_samples_leaf'],
                                         min_samples_split=param_dict[0]['min_samples_split'], 
                                         random_state=0) #RandomForestRegressor(n_estimators = 50, random_state = 0)

    bagging_regressor = BaggingRegressor(estimator=DecisionTreeRegressor(), 
                                         n_estimators=param_dict[1]['n_estimators'],
                                         max_samples=param_dict[1]['max_samples'], 
                                         max_features=param_dict[1]['max_features'],
                                         random_state=42)

    lgbm_regressor = LGBMRegressor(n_estimators=param_dict[2]['n_estimators'], 
                                   max_depth=param_dict[2]['max_depth'],
                                   learning_rate=param_dict[2]['learning_rate'], 
                                   num_leaves=param_dict[2]['num_leaves'],
                                   min_child_samples=param_dict[2]['min_child_samples'], 
                                   random_state=1)

    dt_regressor = DecisionTreeRegressor(max_depth=param_dict[3]['max_depth'],
                                         min_samples_split=param_dict[3]['min_samples_split'],
                                         min_samples_leaf=param_dict[3]['min_samples_leaf'],
                                         random_state=0)

    model = StackingRegressor(
        estimators=[
            ('rf', rf_regressor),
            ('bagging', bagging_regressor),
            ('lgbm', lgbm_regressor),
            ('dt', dt_regressor)],
        final_estimator = LinearRegression()
    )

    model.fit(X_train, y_train)

    predict_train = model.predict(X_train)
    predict_test = model.predict(X_test)

    train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
    test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

    # Concatenate the DataFrames
    combined_df = pd.concat([train_df, test_df])
    combined_df = combined_df.sort_index()
    print(combined_df)

    r2_score_train = round(metrics.r2_score(y_train, predict_train), 2) 
    r2_score_test = round(metrics.r2_score(y_test, predict_test), 2)
    print('r2 train', r2_score_train)    
    print('r2 test', r2_score_test) 
    
    if 'dCn' in filtered_data:
        unit = 'dCn'
    else:
        unit = 'pm_conc'
    
    Scatter_Plot(combined_df, train_df, test_df, r2_score_train, r2_score_test, v, unit,ml_type)
    qq_plot(test_df, v, unit,ml_type)
    return r2_score_test